In [ ]:
import serial
import threading
import time

import sys
sys.path.append("/home/blueberryjam/BlueberryJam/src")
from Serial_connector import *


comm_thread = start_comm_thread(interval=1)

try:
    while True:
        # Here, update your variables as needed
        # Example:
        motor_speed = 200
        servo_angles = [100, 50, 120]
        actuation_times = [1200, 1800, 1600]
        new_shutdown = False
        
        # Update variables in the communication module
        update_variables(motor_speed, servo_angles, new_shutdown, actuation_times)
        
        # Sleep or perform other operations in the main loop
        time.sleep(2)  # Adjust based on your program logic

except KeyboardInterrupt:
    print("Shutting down...")
    shutdown = True
    comm_thread.join()
    ser.close()
    print("Arduino communication stopped.")



Shutting down...


In [5]:
import serial
import threading
import time
import random

# Mock serial communication setup for testing
# Replace '/dev/ttyACM0' with your port if using a real Arduino
port = '/dev/ttyACM0'
ser = serial.Serial(port, 9600, timeout=1) if port else None

# Shared variables
blueberry_process_queue = []
shutdown = False

# Define a mock class to represent blueberries
class Blueberry:
    def __init__(self, belt, ripeness, actuation_time):
        self.belt = belt
        self.ripeness = ripeness
        self.actuation_time = actuation_time

    def __str__(self):
        return f"Belt: {self.belt}, Ripeness: {self.ripeness}, Actuation Time: {self.actuation_time:.3f}"

def send_blueberry_list_data(process_queue, motor_speed=150.0, shutdown=False, update_time=None):
    """
    Sends a data packet for each blueberry in the process queue.
    """
    if update_time is None:
        update_time = time.monotonic()
    elapsed_time = time.monotonic() - update_time
    for berry in process_queue:
        data_packet = f"{motor_speed:.3f}|{berry.belt}|{berry.ripeness}|{int(shutdown)}|{berry.actuation_time:.3f}|{elapsed_time:.3f}|0\n"
        if ser:
            ser.write(data_packet.encode())
        print(f"Sent: {data_packet.strip()}")

def generate_random_blueberries(num=5):
    """
    Generates a random list of blueberries for testing.
    """
    return [
        Blueberry(
            belt=1,
            ripeness=random.choice([-1, 2]),
            actuation_time=random.uniform(1.0, 10.0)
        )
        for _ in range(num)
    ]

def async_blueberry_sender(interval=3):
    """
    Asynchronous thread to send blueberry lists at regular intervals.
    """
    global blueberry_process_queue, shutdown
    while not shutdown:
        if blueberry_process_queue:
            send_blueberry_list_data(blueberry_process_queue)
            blueberry_process_queue.clear()  # Clear after sending
        time.sleep(interval)

# Thread management
def start_blueberry_thread(interval=3):
    """
    Starts the asynchronous blueberry sender thread.
    """
    thread = threading.Thread(target=async_blueberry_sender, args=(interval,), daemon=True)
    thread.start()
    return thread

def stop_threads():
    """
    Stops all running threads.
    """
    global shutdown
    shutdown = True
    time.sleep(1)  # Give threads time to clean up

# Example usage in Jupyter Notebook
def test_blueberry_sending():
    """
    Example function to simulate sending blueberry lists.
    """
    global blueberry_process_queue
    # Generate and update the blueberry queue
    blueberry_process_queue = generate_random_blueberries(num=3)
    print("Generated blueberries:")
    for berry in blueberry_process_queue:
        print(berry)

# Start the thread
shutdown = False
blueberry_thread = start_blueberry_thread(interval=3)

# You can now call `test_blueberry_sending()` in the notebook to add random blueberries to the queue.

# To stop the thread safely, call:
# stop_threads()


Sent: 150.000|1|-1|0|3.799|0.000|0
Sent: 150.000|1|-1|0|6.608|0.000|0


In [10]:
test_blueberry_sending()


Generated blueberries:
Belt: 1, Ripeness: 2, Actuation Time: 1.136
Belt: 1, Ripeness: -1, Actuation Time: 1.225
Belt: 1, Ripeness: -1, Actuation Time: 4.532


Sent: 150.000|1|2|0|1.136|0.000|0
Sent: 150.000|1|-1|0|1.225|0.000|0
Sent: 150.000|1|-1|0|4.532|0.000|0
